# Pinnacle Export Script Example

This notebook demonstrates how to compose a Python script to export raw Pinnacle data to DICOM using the Pinnacle Export Tool in pymedphys.

> Note: If you don't need to export many Pinnacle datasets in bulk, the [Pinnacle Export Tool's command line interface](https://docs.pymedphys.com/en/latest/users/ref/cli/pinnacle.html) may be more suitable.

In [ ]:
import os
import tarfile
import shutil

import tempfile
import urllib
import zipfile

from pathlib import Path

try:
    from pymedphys.pinnacle import PinnacleExport
except ImportError:
    ! pip install pymedphys[user,dicom]
    from pymedphys.pinnacle import PinnacleExport

## Download some test data

In [ ]:
zip_url = "https://zenodo.org/records/3900946/files/pinnacle_test_data.zip?download=1"

output_directory = Path("./data")
with tempfile.TemporaryDirectory() as temp_dir:
    temp_dir = Path(temp_dir)
    temp_file = temp_dir.joinpath("tmp.zip")

    with urllib.request.urlopen(zip_url) as dl_file:
        with open(temp_file, "wb") as out_file:
            out_file.write(dl_file.read())

    with zipfile.ZipFile(temp_file, "r") as zip_ref:
        zip_ref.extractall(output_directory)

## Configure paths

In [ ]:

working_path = os.path.join('pinnacle', 'working') # Directory to extract TAR and export DICOM (should be empty!)
pinn_path = 'Institution_2812/Mount_0/Patient_16218' # Path to Patient directory within TAR
pinn_archive = 'data/pinn/test_pinnacle_16.0.tar.gz' # Path to TAR archive

archive_path = os.path.join(working_path, 'pinn')
export_path = os.path.join(working_path, 'output')

shutil.rmtree(export_path, ignore_errors=True)
os.makedirs(export_path)

print("Files will be exported to: {0}".format(export_path))

## Extract Tar Archive

In [ ]:
t=tarfile.open(pinn_archive)

for m in t.getmembers():
    if ':' not in m.name:
        t.extract(m, path=archive_path)
pinn_extracted = os.path.join(archive_path, pinn_path)

## Initialize Pinnacle object

In [ ]:
pinn = PinnacleExport(pinn_extracted, None)
pinn.logger.setLevel('INFO') # Set the log level to DEBUG to see much more verbose log output

## Find the desired Plan object

In [ ]:
# Find the plan we want to export in the list of plans
plan_name = 'Plan_2'
export_plan = None
for p in pinn.plans:
    if p.plan_info['PlanName'] == plan_name:
        export_plan = p
        break

if not export_plan:
    print("Plan not found. Plans available are: ")
    for p in pinn.plans:
        print(p.plan_info['PlanName'])

## Export primary image of plan

In [ ]:
pinn.export_image(export_plan.primary_image, export_path=export_path)

## Export RTSTRUCT

In [ ]:
pinn.export_struct(export_plan, export_path)

## Export RTPLAN

In [ ]:
pinn.export_plan(export_plan, export_path)

## Export RTDOSE

In [ ]:
pinn.export_dose(export_plan, export_path)

## Export a different image series (not the primary image for the plan)

In [ ]:
series_uid = '1.2.840.113704.1.111.7500.1496636105.13'
for image in pinn.images:
    if series_uid == image.image_header['series_UID']:
        pinn.export_image(image, export_path=export_path)